In [1]:
import os, glob
import numpy as np
import xarray as xr
import pandas as pd
from shapely.geometry import Point
from pyproj import CRS
import geopandas as gpd
import rasterio
from tqdm import tqdm
import datetime as dt

In [2]:
maize_pts = pd.read_excel('/home/rgreen/FoodMarkets/MaizePoints.xls')

In [3]:
maize_pts = maize_pts[['Lon', 'Lat', 'Point_ID']]

In [4]:
#lst_tif_dir = '/home/rgreen/FoodMarkets/Data/MODIS_LST/'
lst_tif_dir = '/home/rgreen/tana-spin/rgreen/FoodMarkets/LST/'

In [34]:
#get all file paths from multiple subdirectories into one list

def file_walk(in_dir, file_ext):
    file_list = []
    for (dirpath, dirnames, filenames) in os.walk(in_dir):
        file_list += [os.path.join(dirpath, file) for file in filenames if file.endswith(file_ext)]
    return sorted(file_list)

In [37]:
lst_files = file_walk(lst_tif_dir, '.tif')

In [40]:
def extract_value_points(file_paths, pts_df, id_col, x_col, y_col):
    
    coords = [(x,y) for x, y in zip(pts_df[x_col], pts_df[y_col])]
    pt_ids = pts_df[id_col]
    pt_vals_df = pd.DataFrame(index = pt_ids)
    pt_vals_df.index.name = None
    
    for file_path in tqdm(file_paths): 
        #print(file_path)
        #extract date from filename in form year+dayofyear
        yr_doy = file_path[file_path.find('CMG_')+4 : file_path.find('.tif')]
        yr, doy = yr_doy[:4], yr_doy[4:]
        
        #convert to date
        date = (pd.to_datetime(int(doy)-1, unit='D', origin=yr)).date()
        
        src = rasterio.open(file_path)
        
        #sample values from list of coordinates
        values = [x[0] for x in src.sample(coords)]
        #set all 0's to nan
        values = [np.nan if x==0 else x for x in values]
        #convert values from kelvin to celsius
        values = [round((0.02 * x - 273.15),2) for x in values]
        pt_vals_df[date] = values

    return pt_vals_df

In [41]:
df = extract_value_points(lst_files, maize_pts, 'Point_ID', 'Lon', 'Lat')

100%|██████████| 6797/6797 [23:31<00:00,  4.81it/s]


In [42]:
df

,2002-07-04,2002-07-05,2002-07-06,2002-07-07,2002-07-08,2002-07-09,2002-07-10,2002-07-11,2002-07-12,2002-07-13,...,2021-02-28,2021-03-01,2021-03-02,2021-03-03,2021-03-04,2021-03-05,2021-03-06,2021-03-07,2021-03-08,2021-03-09
BEN99_0,NaN,30.47,NaN,28.87,NaN,NaN,27.31,27.35,NaN,NaN,...,37.53,42.17,36.27,45.91,NaN,47.01,37.63,43.55,46.97,42.07
BEN99_1,NaN,NaN,NaN,30.17,NaN,NaN,NaN,29.51,32.47,NaN,...,39.47,44.01,37.63,48.31,NaN,49.03,NaN,NaN,48.45,42.55
BEN99_2,NaN,34.31,NaN,34.81,NaN,NaN,NaN,NaN,38.09,NaN,...,40.85,45.13,40.49,51.55,NaN,49.29,39.75,NaN,51.21,46.85
BEN99_3,NaN,29.45,NaN,29.77,NaN,NaN,NaN,29.75,30.61,NaN,...,40.01,44.59,38.71,49.81,NaN,49.99,NaN,NaN,50.67,42.75
BEN99_4,NaN,NaN,NaN,31.09,NaN,NaN,NaN,26.45,31.57,NaN,...,40.95,45.19,38.93,51.93,NaN,49.81,38.85,NaN,50.05,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SDN1554_4577,22.03,24.65,23.17,23.43,25.47,21.55,18.87,23.69,24.93,24.77,...,25.27,27.75,NaN,NaN,27.75,NaN,31.85,26.07,33.75,NaN
SDN1554_4578,23.91,25.29,24.83,23.67,25.95,21.79,19.53,23.09,25.15,27.63,...,23.83,33.09,NaN,26.63,32.95,31.45,36.23,33.63,34.87,NaN
SDN1554_4579,27.17,30.23,28.77,28.71,32.03,26.13,28.31,26.85,29.65,30.97,...,29.11,30.43,32.65,NaN,34.47,33.69,36.53,30.63,36.91,34.55
SDN1554_4580,26.55,29.09,28.41,28.47,30.77,26.79,28.09,26.89,28.69,30.21,...,30.75,31.47,32.11,31.33,37.57,36.51,37.57,24.55,42.43,37.75


In [43]:
#transpose axis
df_T = df.T

In [44]:
df_T.index.name = 'Date'

In [45]:
df_T = df_T.reset_index()

In [46]:
df_T.insert(1,'Year',df_T['Date'].apply(lambda x:x.year))
df_T.insert(2,'Month',df_T['Date'].apply(lambda x:x.month))
df_T.insert(3,'Day',df_T['Date'].apply(lambda x:x.day))

In [47]:
df_T

,Date,Year,Month,Day,BEN99_0,BEN99_1,BEN99_2,BEN99_3,BEN99_4,BEN99_5,...,RWA1400_4572,SDN1554_4573,SDN1554_4574,SDN1554_4575,SDN1554_4576,SDN1554_4577,SDN1554_4578,SDN1554_4579,SDN1554_4580,SDN1554_4581
0,2002-07-04,2002,7,4,NaN,NaN,NaN,NaN,NaN,NaN,...,23.77,21.85,23.69,25.95,21.95,22.03,23.91,27.17,26.55,22.35
1,2002-07-05,2002,7,5,30.47,NaN,34.31,29.45,NaN,NaN,...,25.21,23.41,24.87,27.53,23.69,24.65,25.29,30.23,29.09,23.77
2,2002-07-06,2002,7,6,NaN,NaN,NaN,NaN,NaN,NaN,...,22.59,22.89,22.95,26.41,23.41,23.17,24.83,28.77,28.41,23.13
3,2002-07-07,2002,7,7,28.87,30.17,34.81,29.77,31.09,31.73,...,25.23,23.29,23.73,27.05,23.95,23.43,23.67,28.71,28.47,22.83
4,2002-07-08,2002,7,8,NaN,NaN,NaN,NaN,NaN,NaN,...,24.27,24.55,26.45,29.75,25.55,25.47,25.95,32.03,30.77,25.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6792,2021-03-05,2021,3,5,47.01,49.03,49.29,49.99,49.81,49.19,...,NaN,31.27,30.39,36.47,21.61,NaN,31.45,33.69,36.51,29.41
6793,2021-03-06,2021,3,6,37.63,NaN,39.75,NaN,38.85,38.39,...,27.53,30.15,35.93,39.21,30.77,31.85,36.23,36.53,37.57,32.85
6794,2021-03-07,2021,3,7,43.55,NaN,NaN,NaN,NaN,NaN,...,NaN,31.33,35.45,35.67,32.37,26.07,33.63,30.63,24.55,33.15
6795,2021-03-08,2021,3,8,46.97,48.45,51.21,50.67,50.05,50.27,...,NaN,36.29,38.57,40.17,34.05,33.75,34.87,36.91,42.43,30.29


In [48]:
#save out
df_T.to_csv('/home/rgreen/FoodMarkets/Data/MODIS_LST/' + 'LST_pts.csv')